In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install --upgrade tensorflow

In [ ]:
import pandas as pd
import numpy as np

dataPath = '/content/drive/MyDrive/Soccer Player Injury Prediction/Datasets/Final Dataset/injuryFinalV8582.csv'
df = pd.read_csv(dataPath)
df

,daily_load,fatigue,mood,readiness,sleep_duration,sleep_quality,soreness,stress,injury_ts,weekly_load,...,illness,historical_injury_count,stress_readiness_interaction,team_performance_readiness_interaction,offensive_performance_readiness_interaction,defensive_performance_readiness_interaction,acwr_rolling_avg,stress_rolling_sum,injury,Player_name
0,0.0,3.0,4.0,6.0,8.0,4.0,3.0,0.75,0.0,0.000000,...,0,0,24.0,38.16,41.64,36.84,0.320870,10.50,0,TeamA-3e5f6e2b-46b7-4890-84a9-3bbb2649af5a
1,0.0,3.0,3.0,6.0,7.0,3.0,3.0,0.50,0.0,0.000000,...,0,0,18.0,33.48,30.42,29.88,0.286085,7.00,0,TeamB-4405bb1f-56f7-48ba-bfa8-e795e4006952
2,0.0,3.0,3.0,8.0,8.0,3.0,3.0,0.50,0.0,0.000000,...,0,0,24.0,56.56,56.48,56.72,0.325813,7.00,0,TeamB-48bfd4ea-e9e2-45b5-befc-3383cae87fdf
3,0.0,3.0,3.0,8.0,8.0,3.0,3.0,0.50,0.0,0.000000,...,0,0,24.0,43.36,44.88,51.20,0.272225,7.00,0,TeamB-2f23d7d5-2326-49ce-b9c8-5a6303f785c5
4,200.0,4.0,3.0,8.0,8.0,4.0,3.0,0.50,0.0,0.488536,...,0,0,24.0,56.40,53.92,53.20,0.260681,7.25,0,TeamA-af719df9-3e6c-4ad4-9e8e-0c0c45f4f76a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8577,320.0,2.0,3.0,5.0,7.0,4.0,2.0,0.50,0.0,0.440917,...,0,0,15.0,32.40,27.50,29.15,0.175310,7.00,0,TeamA-4051bba7-1170-4c43-b912-8c38815a7625
8578,0.0,3.0,4.0,6.0,8.0,3.0,2.0,0.25,0.0,0.176367,...,0,0,12.0,40.74,27.60,34.38,0.246456,3.50,0,TeamA-f20565cc-df22-46a7-aa97-af8ed00601d7
8579,0.0,3.0,3.0,7.0,8.0,4.0,3.0,0.50,0.0,0.000000,...,0,0,21.0,46.90,45.57,46.62,0.244719,7.00,0,TeamA-e920ae60-5c4b-4597-be27-fc6876dcec33
8580,0.0,3.0,3.0,8.0,8.0,3.0,3.0,0.50,0.0,0.000000,...,0,0,24.0,57.36,60.96,55.84,0.288546,7.00,0,TeamB-2f23d7d5-2326-49ce-b9c8-5a6303f785c5


In [ ]:
# Print all column names
print("Column Names:")
print(df.columns)

Column Names:
Index(['daily_load', 'fatigue', 'mood', 'readiness', 'sleep_duration',
       'sleep_quality', 'soreness', 'stress', 'injury_ts', 'weekly_load',
       'Team', 'Total_distance', 'Average_running_speed', 'Top_speed', 'HIR',
       'acwr', 'atl', 'ctl28', 'monotony', 'strain', 'team_performance',
       'offensive_performance', 'defensive_performance', 'illness',
       'historical_injury_count', 'stress_readiness_interaction',
       'team_performance_readiness_interaction',
       'offensive_performance_readiness_interaction',
       'defensive_performance_readiness_interaction', 'acwr_rolling_avg',
       'stress_rolling_sum', 'injury', 'Player_name'],
      dtype='object')


In [ ]:
unique_player_names = df['Player_name'].unique().tolist()

# Group by 'Player_name' and check if the maximum value in 'injury' column is 0
players_with_no_injuries = df.groupby('Player_name')['injury'].max() == 0

# Get the list of player names with no injuries
players_with_no_injuries_list = players_with_no_injuries[players_with_no_injuries].index.tolist()

# Drop rows containing those player names
df_filtered = df[~df['Player_name'].isin(players_with_no_injuries_list)]

df = df_filtered

df['injury'].value_counts()

injury
0    3134
1      56
Name: count, dtype: int64

In [ ]:
#Defining class imbalce functions

#Oversampling
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

def oversample_data(X_train, y_train, sampling_ratio=0.35):

    # Apply oversampling on the training data
    sampling_ratio = 0.35
    ros = RandomOverSampler(sampling_strategy=sampling_ratio, random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after oversampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100
    #print(f"Percentage of 0s after oversampling: {zero_percentage_after:.2f}%")

    return X_train_resampled, y_resampled, zero_percentage_after


# Undersampling
def undersample_data(X_train, y_train, sampling_ratio=0.35):

    # Apply undersampling on the training data
    sampling_strategy = 1 - sampling_ratio
    rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after undersampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100

    return X_train_resampled, y_resampled, zero_percentage_after


#SMOTE
def smote_data(X_train, y_train, sampling_ratio):

    # Set a custom sampling ratio
    sampling_ratio = 0.35

    # Apply SMOTE with the custom ratio
    smote = SMOTE(sampling_strategy=sampling_ratio, random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    # Reshape the resampled data
    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after oversampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100


    return X_train_resampled, y_resampled, zero_percentage_after


# ADASYN
def adasyn_oversample_data(X_train, y_train, sampling_ratio=0.35):

    # Apply ADASYN oversampling on the training data
    sampling_strategy = sampling_ratio
    adasyn = ADASYN(sampling_strategy=sampling_strategy, random_state=42, n_neighbors=5)
    X_resampled, y_resampled = adasyn.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after oversampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100


    return X_train_resampled, y_resampled, zero_percentage_after


#Data sample mode
def sample_mode(X_train, y_train, sampling_ratio, oversample_mode):
 precentageOfZeroesInDataset = (y_train == 0).sum() / len(y_train)

 if oversample_mode == 'oversample':
      X_train, y_train, precentageOfZeroesInDataset = oversample_data(X_train, y_train, sampling_ratio)
 elif oversample_mode == 'undersample':
      X_train, y_train, precentageOfZeroesInDataset = undersample_data(X_train, y_train, sampling_ratio)
 elif oversample_mode == 'smote':
      X_train, y_train, precentageOfZeroesInDataset = smote_data(X_train, y_train, sampling_ratio)
 elif oversample_mode == 'adasyn':
      X_train, y_train, precentageOfZeroesInDataset = adasyn_oversample_data(X_train, y_train, sampling_ratio)


 return X_train, y_train, precentageOfZeroesInDataset

In [ ]:
#Confusion matrtix that shows injuries as TP
def confusion_matrix_only_injuries(y_test, y_pred):
    tp, fn, fp, tn = 0, 0, 0, 0
    for i, j in zip(y_test, y_pred):
        if i == 1 and j == 1:
            tp += 1
        elif i == 1 and j == 0:
            fn += 1
        elif i == 0 and j == 1:
            fp += 1
        elif i == 0 and j == 0:
            tn += 1
    # Create a DataFrame for the confusion matrix
    confusion_matrix = pd.DataFrame(
        {
            "Predicted Injuries": [tp, fp],
            "Predicted Non-Injuries": [fn, tn],
        },
        index=["Actual Injuries", "Actual Non-Injuries"]
    )

    return confusion_matrix

In [ ]:
# Datapreprocessing

def create_player_sequences(df, player, sequence_length=7):
    player_df = df[df['Player_name'] == player]
    # player_data = player_df.drop(['Date', 'Player_name'], axis=1).values
    player_data = player_df.drop(['Player_name'], axis=1).values

    sequences = []
    for i in range(len(player_data) - sequence_length):
        sequences.append(player_data[i:i + sequence_length])

    return np.array(sequences)

def create_team_sequence(df, sequence_length):
    players = df['Player_name'].unique()
    all_sequences = []
    for player in players:
        player_sequences = create_player_sequences(df, player, sequence_length)
        all_sequences.append(player_sequences)

    data = np.concatenate(all_sequences, axis=0)
    X, y = data[:, :-1, :], data[:, -1, -1]
    return X, y


def preprocess(inputWindow, interpolate=False):
    # Load your dataframe
    # df = merge_data_for_team("TeamA", interpolate=interpolate)
    df = pd.read_csv(dataPath)

    # # Convert 'Duration' column to seconds
    # df['Duration'] = df['Duration'].apply(convert_duration_to_seconds)

    # Preprocess the data
    X, y = create_team_sequence(df, inputWindow)

    # Convert data types
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    return X, y

In [ ]:
# #LSTM model

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.python.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# def lstm(test_size, oversample_mode, sampling_ratio, epochs = 50):
#     def create_lstm_model(input_shape):
#         model = Sequential()
#         model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
#         model.add(Dropout(0.2))
#         model.add(LSTM(32))
#         model.add(Dropout(0.2))
#         model.add(Dense(1, activation='sigmoid'))

#         model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#         return model

#     input_shape = (X.shape[1], X.shape[2])
#     model = create_lstm_model(input_shape)

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

#     X_train, y_train, precentageOfZeroesInDataset = sample_mode(X_train, y_train, sampling_ratio, oversample_mode)

#     # Train the model
#     history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

#     # Evaluate the model
#     loss, accuracy = model.evaluate(X_test, y_test)

#     # Get the predicted labels
#     y_pred_probs = model.predict(X_test)
#     y_pred = (y_pred_probs > 0.5).astype(int)

#     # Calculate evaluation metrics
#     f1 = f1_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
#     confInjuries = confusion_matrix_only_injuries(y_test, y_pred)

#     return loss, accuracy, f1, precision, recall, confInjuries, precentageOfZeroesInDataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
import numpy as np

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=UserWarning, message="`use_label_encoder` is deprecated in 1.7.0.")

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create a list to store the results
results = []

# List of hyperparameters
input_windows =  [2, 4, 8, 16, 32]
test_sizes = [0.2, 0.3]
oversample_modes = ["none", "oversample", "undersample", "smote", "adasyn"]
sampling_ratios = [0.2, 0.3]
interpolate_injuries = [True, False]

# Iterate through all combinations of hyperparameters
for interploate_injuries in interpolate_injuries:
    for input_window in input_windows:
        X, y = preprocess(input_window, interpolate=interploate_injuries)

        for test_size in test_sizes:
            for oversample_mode in oversample_modes:
                for sampling_ratio in sampling_ratios:

                    input_shape = (X.shape[1], X.shape[2])
                    model = create_lstm_model(input_shape)

                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

                    X_train, y_train, precentageOfZeroesInDataset = sample_mode(X_train, y_train, sampling_ratio, oversample_mode)

                    # Train the model
                    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

                    # Evaluate the model
                    loss, accuracy = model.evaluate(X_test, y_test)

                    # Get the predicted labels
                    y_pred_probs = model.predict(X_test)
                    y_pred = (y_pred_probs > 0.5).astype(int)

                    # Calculate evaluation metrics
                    f1 = f1_score(y_test, y_pred)
                    precision = precision_score(y_test, y_pred)
                    recall = recall_score(y_test, y_pred)
                    #confInjuries = confusion_matrix_only_injuries(y_test, y_pred) # define this function

                    # Save the results
                    results.append({"input_window": input_window,
                                    "interpolate_injuries": interpolate_injuries,
                                    "test_size": test_size,
                                    "oversample_mode": oversample_mode,
                                    "sampling_ratio": sampling_ratio,
                                    "Prectenage of zeroes in dataset": precentageOfZeroesInDataset,
                                    "accuracy": accuracy,
                                    "f1": f1,
                                    "precision": precision,
                                    "recall": recall,
                                    #"confusion": confInjuries
                                    })

# Sort the results based on the F1 score
results.sort(key=lambda x: x['f1'], reverse=True)

# Print the top 3 performers
for i in range(min(3, len(results))):
    print(f"Top Performer {i + 1}:")
    print(f"Input window: {results[i]['input_window']}")
    print(f"Interpolate injuries: {results[i]['interpolate_injuries']}")
    print(f"Test Size: {results[i]['test_size']}")
    print(f"Oversample Mode: {results[i]['oversample_mode']}")
    print(f"Sampling Ratio: {results[i]['sampling_ratio']}")
    print(f"Percentage of zeroes in dataset: {results[i]['Prectenage of zeroes in dataset']}")
    print(f"Accuracy: {results[i]['accuracy']}")
    print(f"F1: {results[i]['f1']}")
    print(f"Precision: {results[i]['precision']}")
    print(f"Recall: {results[i]['recall']}")
    #display(results[i]['confusion'])
    print("\n")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
107/107 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.9414 - loss: 0.3783 - val_accuracy: 0.9936 - val_loss: 0.0428
Epoch 2/2
107/107 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9937 - loss: 0.0407 - val_accuracy: 0.9936 - val_loss: 0.0399
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9953 - loss: 0.0304
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.8007 - loss: 0.4564 - val_accuracy: 0.9936 - val_loss: 0.0432
Epoch 2/2
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9923 - loss: 0.0490 - val_accuracy: 0.9936 - val_loss: 0.0394
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9953 - loss: 0.0301
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Epoch 1/2
118/118 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - accuracy: 0.7440 - loss: 0.4970 - val_accuracy: 0.9263 - val_loss: 0.1931
Epoch 2/2
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.8852 - loss: 0.2709 - val_accuracy: 0.9088 - val_loss: 0.1918
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9032 - loss: 0.1901
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.7926 - loss: 0.4657 - val_accuracy: 0.9825 - val_loss: 0.0909
Epoch 2/2
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 71ms/step - accuracy: 0.8972 - loss: 0.2727 - val_accuracy: 0.9734 - val_loss: 0.0766
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9715 - loss: 0.0717
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
114/114 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - accuracy: 0.7610 - loss: 0.5014 - val_accuracy: 0.9842 - val_loss: 0.0937
Epoch 2/2
114/114 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.8661 - loss: 0.3193 - val_accuracy: 0.9746 - val_loss: 0.1012
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9752 - loss: 0.0968
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Top Performer 1:
Input window: 32
Interpolate injuries: [True, False]
Test Size: 0.3
Oversample Mode: oversample
Sampling Ratio: 0.2
Percentage of zeroes in dataset: 74.08195848855775
Accuracy: 0.9338051676750183
F1: 0.111731843575419
Precision: 0.06097560975609756
Recall: 0.6666666666666666


Top Performer 2:
Input window: 32
Interpolate injuries: [True, False]
Test Size: 0.2
Oversample Mode: oversample
Sampling Ratio: 0.2
Percentage of zeroes in dataset: 74.07450523864959
Accuracy: 0.9588015079498291
F1: 0.10810810810810811
Precision: 0.0625
Recall: 0.4


Top Performer 3:
Input window: 4
Interpolate injuries: [True, Fal